<a href="https://colab.research.google.com/github/kevin36524/LLM_SVDfy/blob/main/LLM_SVDfy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
del model
del tokenizer

In [3]:
# Load model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
#model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
print(sum(p.numel() for p in model.parameters()))

7248023552


In [6]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [ ]:
inputs = tokenizer("Write a poem about Valencia", return_tensors='pt')
print(inputs)
output = model.generate(**inputs, max_length=40)
print(tokenizer.decode(output[0]))

{'input_ids': tensor([[    1, 12786,  1032, 16835,  1452,  4242,  6751]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Write a poem about Valencia, Spain.

Valencia, a city of sun-kissed hues,
Where orange trees bloom and the paella cooks,



In [7]:
from torch import nn
import re

def SVDfy(module: nn.Module, prefix=""):
    for name, child in module._modules.items():
        # Check if the child is a Linear layer
        #isLowerLayer = re.match(r"model\.layers\.([2-3]).*", prefix)
        isLowerLayer = True
        if isinstance(child, nn.Linear) and isLowerLayer:
            print(f"Processing Linear layer: {prefix}{name}")
            W = child.weight.data

            # Derive the dtype from the original weights
            dtype = W.dtype

            #print(f"Original weight shape: {W.shape}")


            # Define the rank
            #rank = 32 if "self_attn" in prefix else 2048
            rank = 32

            # Perform low-rank SVD
            U_r, S_r, Vh_r = torch.svd_lowrank(W, q=rank)

            # Ensure the results are contiguous
            U_r = U_r.to(dtype).contiguous()
            S_r = S_r.to(dtype).contiguous()
            Vh_r = Vh_r.to(dtype).contiguous()

            #print(f"U_r shape: {U_r.shape}")
            #print(f"S_r shape: {S_r.shape}")
            #print(f"Vh_r shape: {Vh_r.shape}")

            # Combine S_r and Vh_r into one matrix
            combined_SVh = torch.mm(torch.diag(S_r), Vh_r.t()).to(dtype).contiguous()

            #print(f"Combined SVh shape: {combined_SVh.shape}")

            # Create the new layers
            u_layer = nn.Linear(U_r.size(0), U_r.size(1), bias=False).to(dtype)  # First layer (Q)
            combined_layer = nn.Linear(combined_SVh.size(0), combined_SVh.size(1), bias=child.bias is not None).to(dtype)  # Second layer (combined S_r * Vh)

            # Assign the weights
            u_layer.weight.data = U_r
            combined_layer.weight.data = combined_SVh

            if child.bias is not None:
                combined_layer.bias.data = child.bias.data.to(dtype)

            # Replace the original linear layer with the new decomposed layers
            module._modules[name] = nn.Sequential(combined_layer, u_layer)

            print(f"Replaced {prefix}{name} with 2 layers")

        # Recursively apply SVD to child modules
        if child is not None:
            SVDfy(child, prefix + name + ".")

# Apply the SVDfy function to the model
SVDfy(model)

Processing Linear layer: model.layers.0.self_attn.q_proj
Replaced model.layers.0.self_attn.q_proj with 2 layers
Processing Linear layer: model.layers.0.self_attn.k_proj
Replaced model.layers.0.self_attn.k_proj with 2 layers
Processing Linear layer: model.layers.0.self_attn.v_proj
Replaced model.layers.0.self_attn.v_proj with 2 layers
Processing Linear layer: model.layers.0.self_attn.o_proj
Replaced model.layers.0.self_attn.o_proj with 2 layers
Processing Linear layer: model.layers.0.mlp.gate_proj
Replaced model.layers.0.mlp.gate_proj with 2 layers
Processing Linear layer: model.layers.0.mlp.up_proj
Replaced model.layers.0.mlp.up_proj with 2 layers
Processing Linear layer: model.layers.0.mlp.down_proj
Replaced model.layers.0.mlp.down_proj with 2 layers
Processing Linear layer: model.layers.1.self_attn.q_proj
Replaced model.layers.1.self_attn.q_proj with 2 layers
Processing Linear layer: model.layers.1.self_attn.k_proj
Replaced model.layers.1.self_attn.k_proj with 2 layers
Processing Lin

In [8]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Sequential(
            (0): Linear(in_features=32, out_features=4096, bias=False)
            (1): Linear(in_features=4096, out_features=32, bias=False)
          )
          (k_proj): Sequential(
            (0): Linear(in_features=32, out_features=4096, bias=False)
            (1): Linear(in_features=1024, out_features=32, bias=False)
          )
          (v_proj): Sequential(
            (0): Linear(in_features=32, out_features=4096, bias=False)
            (1): Linear(in_features=1024, out_features=32, bias=False)
          )
          (o_proj): Sequential(
            (0): Linear(in_features=32, out_features=4096, bias=False)
            (1): Linear(in_features=4096, out_features=32, bias=False)
          )
          (rotary_emb): MistralRotaryEmbedding()
        )


In [9]:
print(sum(p.numel() for p in model.parameters()))

219549696


In [10]:
inputs = tokenizer("Write a poem about Valencia", return_tensors='pt')
print(inputs)
output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0]))

{'input_ids': tensor([[    1, 12786,  1032, 16835,  1452,  4242,  6751]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Write a poem about Valencia                                                                                             


In [ ]:
def make_sequential_contiguous(module: nn.Module):
    for name, child in module.named_children():
        if isinstance(child, nn.Sequential):
            for layer in child:
                if isinstance(layer, nn.Linear):
                    layer.weight.data = layer.weight.data.contiguous()
                    if layer.bias is not None:
                        layer.bias.data = layer.bias.data.contiguous()
            print(f"Converted Sequential block {name} to contiguous memory.")

        # Recursively apply to all sub-modules
        make_sequential_contiguous(child)

make_sequential_contiguous(model)

In [12]:

# Save and push the model to Hugging Face hub
model.push_to_hub("kevin36524/SVDfy_Mistral-7B_r32")

# Save and push the tokenizer to Hugging Face hub
tokenizer.push_to_hub("kevin36524/SVDfy_Mistral-7B_r32")


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/878M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kevin36524/SVDfy_Mistral-7B_r32/commit/bdce9e39f5de061d388428144542f6d8bc652790', commit_message='Upload tokenizer', commit_description='', oid='bdce9e39f5de061d388428144542f6d8bc652790', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print(model)

In [ ]:
# Save the modified model
output_dir = "./modified_model"  # Specify your output directory
model.save_pretrained(output_dir)

# Save the tokenizer if needed
tokenizer.save_pretrained(output_dir)